## Classic class structure

In [8]:
from abc import ABC, abstractmethod
from collections import namedtuple

----

In [9]:
Customer = namedtuple('Customer', 'name fidelity')

In [10]:
class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity

In [11]:
class Order: # the context
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


----

In [6]:
class Promotion(ABC): # the strategy: an abstract base class
    
    @abstractmethod
    def discount(self, order):
        """Return discount as a positive dollar amount"""

In [7]:
class FidelityPromo(Promotion): # first concrete strategy
    """5% discount for customers with 1000 or more fidelity points"""
    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0
    

In [8]:
class BulkItemPromo(Promotion):
    """10% discount for each LineItem with 20 or more units"""
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1
        return discount

In [9]:
class LargeOrderPromo(Promotion):
    """7% discount for orders with 10 or more distinct items"""
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0

### Test cases

In [13]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

In [14]:
cart = [LineItem('banana', 4, 0.5),
        LineItem('apple', 10, 1.5),
        LineItem('watermelon', 5, 5.0)]

In [13]:
Order(joe, cart, FidelityPromo())

<Order total: 42.000000 due: 42.000000>

In [14]:
Order(ann, cart, FidelityPromo())

<Order total: 42.000000 due: 39.900000>

In [27]:
banana_cart = [LineItem('banana', 30, 0.5),
               LineItem('apple', 10, 1.5)]

In [16]:
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.000000 due: 28.500000>

In [28]:
long_order = [LineItem(str(item_code), 1, 1.0)
              for item_code in range(10)]

In [20]:
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.000000 due: 9.300000>

In [21]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.000000 due: 42.000000>

## Functional approach

In [22]:
def fidelity_promo(order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

In [24]:
def bulk_item_promo(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

In [25]:
def large_order_promo(order):
    discount_items = {item.product for item in order.cart}
    if len(discount_items) >= 10:
        return order.total() * 0.07
    return 0

### Test cases

In [28]:
Order(joe, cart, fidelity_promo)

<Order total: 42.000000 due: 42.000000>

In [30]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.000000 due: 28.500000>

In [34]:
Order(joe, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [35]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]
def best_promo(order):
    """select best discount available"""
    return max(promo(order) for promo in promos)

In [36]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [37]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [38]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

In [39]:
promos = [globals()[name] for name in globals()
          if name.endswith('_promo')
          and name != 'best_promo']

In [40]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

# Decorator strategy pattern

In [20]:
promos = []

In [2]:
def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

In [21]:
@promotion
def fidelity(order):
    '''5% discount for customers with 1000 or more fidelity points'''
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

In [22]:
@promotion
def bulk_item(order):
    '''10% discount for each LineItem with 20 or more units'''
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

In [23]:
@promotion
def large_order(order):
    '''7% discount for orders with 10 or more discount items'''
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

In [7]:
def best_promo(order):
    '''select best discount available'''
    return max(promo(order) for promo in promos)

In [24]:
promos

[<function __main__.fidelity>,
 <function __main__.bulk_item>,
 <function __main__.large_order>]

In [29]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

----